## Import Necessary Libraries

In [120]:
import os
import shutil
import torch
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import cv2
import json
import random


# Set plot style for Seaborn
sns.set(style="darkgrid")

# Set the display options for pandas
pd.set_option('display.max_colwidth', 200)
pd.set_option('display.max_rows', None)

## Clean Working Directory

In [1]:

# Path to the working directory
working_dir = "/kaggle/working/"

# Remove all files and subdirectories
try:
    shutil.rmtree(working_dir)
    os.makedirs(working_dir)  # Recreate the directory if needed
    print(f"Cleared the working directory: {working_dir}")
except Exception as e:
    print(f"Error clearing working directory: {e}")

Error clearing working directory: [Errno 16] Device or resource busy: '/kaggle/working/'


In [61]:
# Define the paths to the directories
directories_to_delete = [
    '/kaggle/working/PEDESTRIAN'
]

# Loop through the list of directories and delete them if they exist
for dir_path in directories_to_delete:
    if os.path.exists(dir_path):
        shutil.rmtree(dir_path)  # Use shutil.rmtree to remove the directory and its contents
        print(f"{dir_path} has been deleted.")
    else:
        print(f"{dir_path} does not exist.")


/kaggle/working/PEDESTRIAN has been deleted.


In [ ]:
# Define the paths to the files
files_to_delete = [
    '/kaggle/working/PEDESTRIAN.zip',
]

# Loop through the list of files and delete them if they exist
for file_path in files_to_delete:
    if os.path.exists(file_path):
        os.remove(file_path)
        print(f"{file_path} has been deleted.")
    else:
        print(f"{file_path} does not exist.")

In [3]:
# Set the device to cuda if available, otherwise cpu
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

## EDA

In [4]:
# Paths
images_path = "/kaggle/input/pedestrian-iit-delhi/Dataset_for_CV_intern_Assignment/Pedestrian_dataset_for_internship_assignment/Pedestrian_dataset_for_internship_assignment"
annotations_path = "/kaggle/input/pedestrian-iit-delhi/Dataset_for_CV_intern_Assignment/random_sample_mavi_2_gt.json"

In [64]:
# Open the JSON file
with open(annotations_path, 'r') as file:
    data = json.load(file) 

In [65]:
data.keys()

dict_keys(['images', 'annotations', 'categories'])

In [66]:
data['images'][0]

{'height': 480, 'width': 640, 'id': 1, 'file_name': '1229.jpg'}

In [9]:
data['annotations'][0]

{'iscrowd': 0,
 'image_id': 1,
 'bbox': [297, 262, 12, 33],
 'segmentation': [],
 'category_id': 1,
 'id': 1,
 'area': 396,
 'ignore': 0,
 'vis_ratio': 1.0,
 'height': 33}

In [10]:
data['categories'][0]

{'id': 1, 'name': 'pedestrian', 'supercategory': 'pedestrian'}

In [ ]:
# Pretty print the 'images' section
if 'images' in data:
    i=0
    print("\nImages:")
    for image in data['images']:
        i+=1
        print(f"  Image ID: {image['id']}")
        print(f"  File Name: {image['file_name']}")
        print(f"  Dimensions: {image['width']}x{image['height']}")
        print()
    print(f"Count:", i)

In [ ]:
# Pretty print the 'annotations' section
if 'annotations' in data:
    i=0
    print("\nAnnotations:")
    for annotation in data['annotations']:
        i+=1
        print(f"  Image ID: {annotation['image_id']}")
        print(f"  Bounding Box: {annotation['bbox']}")
        print(f"  Category ID: {annotation['category_id']}")
        print() 
    print(f"Count:", i)

## Split dataset into Train and Val sets

In [62]:
# Define the paths
original_images_path = "/kaggle/input/pedestrian-iit-delhi/Dataset_for_CV_intern_Assignment/Pedestrian_dataset_for_internship_assignment/Pedestrian_dataset_for_internship_assignment"
original_annotations_path = "/kaggle/input/pedestrian-iit-delhi/Dataset_for_CV_intern_Assignment/random_sample_mavi_2_gt.json"
output_dir = "/kaggle/working/pedestrian"

# Create the necessary directories
train_images_dir = os.path.join(output_dir, "train")
val_images_dir = os.path.join(output_dir, "val")
annotations_dir = os.path.join(output_dir, "annotations")

os.makedirs(train_images_dir, exist_ok=True)
os.makedirs(val_images_dir, exist_ok=True)
os.makedirs(annotations_dir, exist_ok=True)

# Load the original annotations
with open(original_annotations_path, 'r') as file:
    data = json.load(file)

# Get the list of images
images = data['images']
random.shuffle(images)  # Shuffle the images to split randomly

# Split images into training and validation sets
train_images = images[:160]
val_images = images[160:200]

# Create a mapping from image_id to file_name for quick access
image_id_to_file = {img['id']: img['file_name'] for img in images}

# Prepare the annotation data for train and val
train_annotations = []
val_annotations = []

for annotation in data['annotations']:
    if annotation['image_id'] in [img['id'] for img in train_images]:
        train_annotations.append(annotation)
    elif annotation['image_id'] in [img['id'] for img in val_images]:
        val_annotations.append(annotation)

# Copy images to train and val folders
for img in train_images:
    src_path = os.path.join(original_images_path, img['file_name'])
    dst_path = os.path.join(train_images_dir, img['file_name'])
    shutil.copy(src_path, dst_path)

for img in val_images:
    src_path = os.path.join(original_images_path, img['file_name'])
    dst_path = os.path.join(val_images_dir, img['file_name'])
    shutil.copy(src_path, dst_path)

# Create the final annotation JSON files
instances_train = {
    'images': train_images,
    'annotations': train_annotations,
    'categories': data['categories']
}

instances_val = {
    'images': val_images,
    'annotations': val_annotations,
    'categories': data['categories']
}

# Save the annotations as JSON files
with open(os.path.join(annotations_dir, 'instances_train.json'), 'w') as f:
    json.dump(instances_train, f)

with open(os.path.join(annotations_dir, 'instances_val.json'), 'w') as f:
    json.dump(instances_val, f)

print("Dataset has been successfully split and organized!")


Dataset has been successfully split and organized!


In [63]:
# Define the path to the PEDESTRIAN folder
pedestrian_path = "/kaggle/working/pedestrian"
zip_file_path = "/kaggle/working/pedestrian.zip"

# Create a ZIP file of the COCODIR folder
shutil.make_archive(zip_file_path[:-4], 'zip', pedestrian_path)

print("ZIP file created successfully.")


ZIP file created successfully.


## Clone DINO repository

In [13]:
!git clone https://github.com/IDEA-Research/DINO.git

Cloning into 'DINO'...
remote: Enumerating objects: 442, done.
remote: Counting objects: 100% (191/191), done.
remote: Compressing objects: 100% (95/95), done.
remote: Total 442 (delta 136), reused 96 (delta 96), pack-reused 251 (from 1)
Receiving objects: 100% (442/442), 13.43 MiB | 20.47 MiB/s, done.
Resolving deltas: 100% (191/191), done.


In [14]:
%cd DINO

/kaggle/working/DINO


## Setting up the necessary environment and dependencies

In [15]:
!conda install -c pytorch pytorch torchvision -y

Retrieving notices: ...working... done
Channels:
 - pytorch
 - rapidsai
 - nvidia
 - nodefaults
 - conda-forge
 - defaults
Platform: linux-64
Solving environment: done

## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - pytorch
    - torchvision


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    blas-1.0                   |              mkl           1 KB  conda-forge
    ca-certificates-2024.8.30  |       hbcca054_0         155 KB  conda-forge
    certifi-2024.8.30          |     pyhd8ed1ab_0         160 KB  conda-forge
    cuda-cccl-12.4.127         |                0         1.4 MB  nvidia
    cuda-cudart-12.4.127       |                0         198 KB  nvidia
    cuda-cudart-dev-12.4.127   |                0         413 KB  nvidia
    cuda-cupti-12.4.127        |                0        16.4 MB  nvidia
    cuda-libraries-12.4.0      |              

In [16]:
!pip install -r requirements.txt

  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-install-vvgrbpvp/pycocotools_482c1cb20c9d48848d349c73008332a7
  Running command git clone --filter=blob:none --quiet https://github.com/cocodataset/cocoapi.git /tmp/pip-install-vvgrbpvp/pycocotools_482c1cb20c9d48848d349c73008332a7
  Resolved https://github.com/cocodataset/cocoapi.git to commit 8c9bcc3cf640524c4c20a9c40e89cb6a2f2fa0e9
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/cocodataset/panopticapi.git to /tmp/pip-install-vvgrbpvp/panopticapi_8550b083170f4e70b72d5ea873d7bc2c
  Running command git clone --filter=blob:none --quiet https://github.com/cocodataset/panopticapi.git /tmp/pip-install-vvgrbpvp/panopticapi_8550b083170f4e70b72d5ea873d7bc2c
  Resolved https://github.com/cocodataset/panopticapi.git to commit 7bb4655548f98f3fedc07bf37e9040a992b054b0
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.9/74.9 kB 3.8 MB/s eta 0:00:00
  Created wheel for pycoco

In [17]:
# Change to the directory
%cd models/dino/ops

/kaggle/working/DINO/models/dino/ops


In [18]:
# Build and install the package
!python setup.py build install

/opt/conda/lib/python3.10/site-packages/torch/utils/cpp_extension.py:414: UserWarning: The detected CUDA version (12.3) has a minor version mismatch with the version that was used to compile PyTorch (12.4). Most likely this shouldn't be a problem.
  warnings.warn(CUDA_MISMATCH_WARN.format(cuda_str_version, torch.version.cuda))
/opt/conda/lib/python3.10/site-packages/torch/utils/cpp_extension.py:424: UserWarning: There are no g++ version bounds defined for CUDA version 12.3
  warnings.warn(f'There are no {compiler_name} version bounds defined for CUDA version {cuda_str_version}')
/opt/conda/lib/python3.10/site-packages/torch/utils/cpp_extension.py:1965: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(
Emitting ninja build file /kaggle/working/DINO/models/dino/ops/build/temp.linux-x86_64-cpython-310/build.ninja...
Compiling objects...
Allowing ni

In [19]:
# Run the unit tests
!python test.py

* True check_forward_equal_with_pytorch_double: max_abs_err 8.67e-19 max_rel_err 2.35e-16
* True check_forward_equal_with_pytorch_float: max_abs_err 4.66e-10 max_rel_err 1.13e-07
* True check_gradient_numerical(D=30)
* True check_gradient_numerical(D=32)
* True check_gradient_numerical(D=64)
* True check_gradient_numerical(D=71)
* True check_gradient_numerical(D=1025)
Traceback (most recent call last):
  File "/kaggle/working/DINO/models/dino/ops/test.py", line 86, in <module>
    check_gradient_numerical(channels, True, True, True)
  File "/kaggle/working/DINO/models/dino/ops/test.py", line 76, in check_gradient_numerical
    gradok = gradcheck(func, (value.double(), shapes, level_start_index, sampling_locations.double(), attention_weights.double(), im2col_step))
  File "/opt/conda/lib/python3.10/site-packages/torch/autograd/gradcheck.py", line 2053, in gradcheck
    return _gradcheck_helper(**args)
  File "/opt/conda/lib/python3.10/site-packages/torch/autograd/gradcheck.py", line 208

In [98]:
# Change back to the original directory
%cd ../../..

/


In [21]:
!pip install yapf==0.32.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.2/190.2 kB 11.7 MB/s eta 0:00:00
  Attempting uninstall: yapf
    Found existing installation: yapf 0.40.2
    Uninstalling yapf-0.40.2:
      Successfully uninstalled yapf-0.40.2


## Handle NumPy attribute error

In [73]:
# Define the path to the file
file_path = '/opt/conda/lib/python3.10/site-packages/pycocotools/cocoeval.py'

# Read the file, replace np.float with float, and write it back
with open(file_path, 'r') as file:
    file_contents = file.read()


# Replace np.float with float
updated_contents = file_contents.replace('np.float', 'float')

# Write the updated contents back to the file
with open(file_path, 'w') as file:
    file.write(updated_contents)

print("File updated successfully.")

File updated successfully.


## Performing evaluations on validation set

In [74]:
!bash scripts/DINO_eval.sh /kaggle/input/pedestrian-dataset/coco2017 /kaggle/input/dino_4scale/other/default/1/checkpoint0011_4scale.pth

Not using distributed mode
Loading config file from config/DINO/DINO_4scale.py
[09/24 09:30:16.685]: git:
  sha: d84a491d41898b3befd8294d1cf2614661fc0953, status: clean, branch: main

[09/24 09:30:16.685]: Command: main.py --output_dir logs/DINO/R50-MS4-%j -c config/DINO/DINO_4scale.py --coco_path /kaggle/input/pedestrian-dataset/coco2017 --eval --resume /kaggle/input/dino_4scale/other/default/1/checkpoint0011_4scale.pth --options dn_scalar=100 embed_init_tgt=TRUE dn_label_coef=1.0 dn_bbox_coef=1.0 use_ema=False dn_box_noise_scale=1.0
[09/24 09:30:16.686]: Full config saved to logs/DINO/R50-MS4-%j/config_args_all.json
[09/24 09:30:16.686]: world size: 1
[09/24 09:30:16.687]: rank: 0
[09/24 09:30:16.687]: local_rank: 0
[09/24 09:30:16.687]: args: Namespace(config_file='config/DINO/DINO_4scale.py', options={'dn_scalar': 100, 'embed_init_tgt': True, 'dn_label_coef': 1.0, 'dn_bbox_coef': 1.0, 'use_ema': False, 'dn_box_noise_scale': 1.0}, dataset_file='coco', coco_path='/kaggle/input/pedest

In [99]:
# AP values obtained from the evaluation
ap_values = {
    'AP IoU=0.50:0.95': 0.496,
    'AP IoU=0.50': 0.832,
    'AP IoU=0.75': 0.534,
    'AP (small area)': 0.394,
    'AP (medium area)': 0.633,
    'AP (large area)': 0.718,
}

# Create a DataFrame for better visualization
ap_df = pd.DataFrame(list(ap_values.items()), columns=['Metric', 'Value'])
print(ap_df)

             Metric  Value
0  AP IoU=0.50:0.95  0.496
1       AP IoU=0.50  0.832
2       AP IoU=0.75  0.534
3   AP (small area)  0.394
4  AP (medium area)  0.633
5   AP (large area)  0.718


In [110]:
!pip install --upgrade torch torchvision


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.1/797.1 MB 2.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 84.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 71.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 8.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 13.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 8.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.2/176.2 MB 9.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━

In [127]:
from models.dino import build_dino_model

AttributeError: partially initialized module 'torchvision' has no attribute 'extension' (most likely due to a circular import)

In [126]:
# Directories for images and annotations
val_images_dir = "/kaggle/input/pedestrian-dataset/coco2017/val2017"
annotations_dir = "/kaggle/input/pedestrian-dataset/coco2017/annotations"

# Load the COCO-style annotations
with open(os.path.join(annotations_dir, 'instances_val2017.json')) as f:
    val_data = json.load(f)

# Load your DINO model (update with your actual model loading logic)
from models.dino import build_dino_model  # Adjust import as per your structure
model = build_dino_model()
checkpoint = torch.load('/kaggle/input/dino_4scale/other/default/1/checkpoint0011_4scale.pth')
model.load_state_dict(checkpoint)
model.eval()  # Set the model to evaluation mode
model.to(device)  # Move model to GPU

# Prepare an empty list to hold predictions
results = []

# Loop through each validation image
for img in val_data['images']:
    image_id = img['id']
    img_path = os.path.join(val_images_dir, img['file_name'])
    
    # Load the image
    image = cv2.imread(img_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Preprocess the image (resizing, normalization, etc.)
    input_image = cv2.resize(image_rgb, (640, 640))  # Adjust to DINO input size if necessary
    input_image = np.transpose(input_image, (2, 0, 1))  # Change to (C, H, W)
    input_image = input_image / 255.0  # Normalize to [0, 1]
    input_tensor = torch.tensor(input_image, dtype=torch.float32).unsqueeze(0).to(device)  # Add batch dimension

    # Perform inference
    with torch.no_grad():
        outputs = model(input_tensor)  # Get model predictions
        # Assuming outputs contains 'boxes' and 'scores' in a compatible format
        boxes = outputs['boxes'].cpu().numpy()
        scores = outputs['scores'].cpu().numpy()

    # Threshold for predictions
    threshold = 0.5
    for i, score in enumerate(scores):
        if score > threshold:
            x1, y1, x2, y2 = boxes[i]  # Adjust based on your box output format
            width = x2 - x1
            height = y2 - y1
            results.append({
                'image_id': image_id,
                'bbox': [x1, y1, width, height],
                'score': score
            })

# Function to visualize detections
def visualize_detections(image_id, predictions, ground_truths, img_dir):
    # Load the image
    image_path = os.path.join(img_dir, image_id)
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Draw ground truth boxes in green
    for gt in ground_truths:
        if gt['image_id'] == image_id:
            x, y, w, h = gt['bbox']
            cv2.rectangle(image_rgb, (int(x), int(y)), (int(x + w), int(y + h)), (0, 255, 0), 2)

    # Draw predicted boxes in red
    for pred in predictions:
        if pred['image_id'] == image_id:
            x, y, w, h = pred['bbox']
            cv2.rectangle(image_rgb, (int(x), int(y)), (int(x + w), int(y + h)), (255, 0, 0), 2)

    plt.figure(figsize=(10, 10))
    plt.imshow(image_rgb)
    plt.axis('off')
    plt.title(f"Detections for {image_id}")
    plt.show()

# Visualize detection results for a few random images
sample_images = random.sample(val_data['images'], 5)  # Get 5 random images from the validation set

for img in sample_images:
    image_id = img['file_name']
    ground_truths = [ann for ann in val_data['annotations'] if ann['image_id'] == img['id']]
    predictions_for_image = [pred for pred in results if pred['image_id'] == img['id']]
    
    visualize_detections(image_id, predictions_for_image, ground_truths, val_images_dir)


AttributeError: partially initialized module 'torchvision' has no attribute 'extension' (most likely due to a circular import)